# Overview

This notebook is used for the development of the new orientation class that will function in place of FZfile.py and RotRep.

In [1]:
# import the functions that need to be migrated to the new classes

import os
import sys
import numpy as np

from tqdm import tqdm_notebook
from scipy.linalg import polar

sys.path.append("../../")

In [2]:
from RotRep import Mat2EulerZXZ, Mat2EulerZXZVectorized 
from RotRep import EulerZXZ2Mat, EulerZXZ2MatVectorized
from RotRep import rod_from_quaternion
from RotRep import Misorien2FZ1
from RotRep import GetSymRotMat
from RotRep import Orien2FZ
# from FZfile import generate_random_rot_mat

# from FZfile import misorien

In [3]:
from hexomap.orientation import Eulers
from hexomap.orientation import Quaternion
from hexomap.orientation import Rodrigues
from hexomap.orientation import Orientation
from hexomap.orientation import Frame
from hexomap.orientation import sym_operator
from hexomap.utility     import iszero
from hexomap.utility     import isone
from hexomap.npmath      import random_three_vector
from hexomap.lattice     import to_fundamental_zone

The FZfile is somehow tied to pycuda, we have to come back for it later.

# Euler angles and (active) rotation matrix conversion

The previous implementation is done directly between the two, now we need to add quaternion to the fold.

In [ ]:
EulerZXZ2Mat??

In [ ]:
euler = [-1.26124505, -1.34263514, 0.39034601]
print(EulerZXZ2Mat(euler))

In [ ]:
from hexomap.orientation import Eulers

In [ ]:
e = Eulers(*euler)
print(e)
print(e.as_matrix)

In [ ]:
# quick test

eulers = ((np.random.random(100*3)-0.5)*2*np.pi).reshape(100, 3)

for me in eulers:
    try:
        np.testing.assert_allclose(EulerZXZ2Mat(me), Eulers(*me).as_matrix)
    except:
        print("error")
        print(me)
        print(EulerZXZ2Mat(me))
        print(Eulers(*me).as_matrix)


In [ ]:
EulerZXZ2MatVectorized??

In [ ]:
N = 10000

eulers = ((np.random.random(N*3)-0.5)*2*np.pi).reshape(N, 3)

In [ ]:
%%timeit

m_old = EulerZXZ2MatVectorized(eulers)

In [ ]:
%%timeit

m_new = Eulers.eulers_to_matrices(eulers)

In [ ]:
np.testing.assert_allclose(m_old, m_new)

In [ ]:
Mat2EulerZXZ??

In [ ]:
# original method
# euler -> matrix -> euler

euler = np.radians([0, 0, 32])  # special case, also need to make sure that the single rotation is the last one.
m = EulerZXZ2Mat(euler)

np.testing.assert_allclose(euler, Mat2EulerZXZ(m))

In [ ]:
# new method

euler = Eulers(*np.radians([0, 0, 32]))

m = euler.as_matrix

print(Eulers.from_matrix(m))
print(euler)

Both methods are self consistent, now need to make sure that both can get the same Euler angles from a random rotation matrix

In [ ]:
F = np.random.random(3*3).reshape(3,3)

R, V = polar(F)

print(R)
print(V)

In [ ]:
euler_o = Mat2EulerZXZ(R)

euler_n = Eulers.from_matrix(R).as_array

print(np.array(euler_o)%(2*np.pi), euler_n)

In [ ]:
Mat2EulerZXZVectorized??

In [ ]:
Fs = np.random.random(5*3*3).reshape(5,3,3)
Rs = np.array([polar(me)[0] for me in Fs])

eulers_o = Mat2EulerZXZVectorized(Rs)

print(eulers_o)

In [ ]:
eulers_n = Eulers.matrices_to_eulers(Rs)

print(eulers_n)

In [ ]:
# now batch testing

N = 10000
Fs = np.random.random(N*3*3).reshape(N,3,3)
Rs = np.array([polar(me)[0] for me in Fs])

In [ ]:
%%timeit
eulers_o = Mat2EulerZXZVectorized(Rs)

In [ ]:
%%timeit
eulers_n = Eulers.matrices_to_eulers(Rs)

In [ ]:
np.testing.assert_allclose(eulers_o, eulers_n)

# Quaternion dev

## Quaternion <-> Euler angles

In [ ]:
euler = np.radians([33, 10, 1])

e = Eulers(*euler)
q = Quaternion.from_eulers(e)

print(np.degrees(q.as_eulers.as_array))

In [ ]:
for _ in tqdm_notebook(range(10000)):
    euler = Eulers(*((np.random.random(3)-0.5)*2*np.pi))
    q = Quaternion.from_eulers(euler)
    np.testing.assert_allclose(euler.as_array, q.as_eulers.as_array)

In [ ]:
P = 1

def eu2qu(eu):
    """Euler angles to quaternion"""
    ee = 0.5*eu
    cPhi = np.cos(ee[1])
    sPhi = np.sin(ee[1])
    qu = np.array([ cPhi*np.cos(ee[0]+ee[2]),
                   -P*sPhi*np.cos(ee[0]-ee[2]),
                   -P*sPhi*np.sin(ee[0]-ee[2]),
                   -P*cPhi*np.sin(ee[0]+ee[2]) ])
    #if qu[0] < 0.0: qu.homomorph()                                                                   !ToDo: Check with original
    return qu

def qu2eu(qu):
    """Quaternion to Euler angles""" 
    q03 = qu[0]**2+qu[3]**2
    q12 = qu[1]**2+qu[2]**2
    chi = np.sqrt(q03*q12)
    
    if iszero(chi):
        eu = np.array([np.arctan2(-P*2.0*qu[0]*qu[3],qu[0]**2-qu[3]**2), 0.0,   0.0]) if iszero(q12) else \
        np.array([np.arctan2(2.0*qu[1]*qu[2],qu[1]**2-qu[2]**2),         np.pi, 0.0])
    else:
        eu = np.array([np.arctan2((-P*qu[0]*qu[2]+qu[1]*qu[3])*chi, (-P*qu[0]*qu[1]-qu[2]*qu[3])*chi ),
                       np.arctan2( 2.0*chi, q03-q12 ), 
                       np.arctan2(( P*qu[0]*qu[2]+qu[1]*qu[3])*chi, (-P*qu[0]*qu[1]+qu[2]*qu[3])*chi )])
    
    # reduce Euler angles to definition range, i.e a lower limit of 0.0
    eu = np.where(eu<0, (eu+2.0*np.pi)%np.array([2.0*np.pi,np.pi,2.0*np.pi]),eu)
    return eu

for _ in tqdm_notebook(range(10000)):
    eu = (np.random.random(3)-0.5)*np.pi*2
    eu = np.where(eu<0, (eu+2.0*np.pi)%np.array([2.0*np.pi,np.pi,2.0*np.pi]),eu%(2*np.pi))
    q = eu2qu(eu)
    np.testing.assert_allclose(eu, qu2eu(q))

## Quaternion <-> Rotation Matrix

In [ ]:
for _ in tqdm_notebook(range(10000)):
    euler = Eulers(*((np.random.random(3)-0.5)*2*np.pi))
    m = euler.as_matrix
    
    q = Quaternion.from_matrix(m)
    np.testing.assert_allclose(m, q.as_matrix)

## Rodrigues <-> Quaternion

In [ ]:
rod_from_quaternion??

In [ ]:
a = np.ones((5,5))
b = np.arange(5) + 1

print(a/b[:,None])

In [ ]:
qs = np.array([Quaternion.from_random().as_array for _ in range (5)]).T

In [ ]:
print(qs)

In [ ]:
rs_o = rod_from_quaternion(qs)

print(rs_o)

In [ ]:
rs_n = Rodrigues.rodrigues_from_quaternions(qs.T).T

print(rs_n)

# Misorientation related calculation

In [11]:
Misorien2FZ1??

In [9]:
q1 = Quaternion.from_angle_axis(np.pi/4,         axis=np.array([0,1,1]))
q2 = Quaternion.from_angle_axis(np.pi/4+np.pi/3, axis=np.array([0,1,1]))

frame_lab = Frame()

o1 = Orientation(q1, frame_lab)
o2 = Orientation(q2, frame_lab)

print(q1)
print(q2)
print(frame_lab)
print(o1)
print(o2)

_ang, _ = o1.misorientation(o2, 'hcp')

print(np.degrees(_ang))

Quaternion(w=0.9238795325112868, x=0.0, y=0.2705980500730985, z=0.2705980500730985, normalized=True)
Quaternion(w=0.6087614290087208, x=0.0, y=0.5609855267969309, z=0.5609855267969309, normalized=True)
Frame(e1=array([1, 0, 0]), e2=array([0, 1, 0]), e3=array([0, 0, 1]), o=array([0, 0, 0]), name='lab')
Orientation(q=Quaternion(w=0.9238795325112868, x=0.0, y=0.2705980500730985, z=0.2705980500730985, normalized=True), f=Frame(e1=array([1, 0, 0]), e2=array([0, 1, 0]), e3=array([0, 0, 1]), o=array([0, 0, 0]), name='lab'))
Orientation(q=Quaternion(w=0.6087614290087208, x=0.0, y=0.5609855267969309, z=0.5609855267969309, normalized=True), f=Frame(e1=array([1, 0, 0]), e2=array([0, 1, 0]), e3=array([0, 0, 1]), o=array([0, 0, 0]), name='lab'))
44.124382315082926


In [10]:
frame_lab = Frame()

axis = random_three_vector()
q_0 = Quaternion.from_angle_axis(0, axis)
o_0 = Orientation(q_0, frame_lab)

for me in [None, 'bcc', 'hcp']:
    for _ in tqdm_notebook(range(100), desc=f"{me}"):
        ang = (np.random.random())*np.pi/4
        o_i = Orientation(Quaternion.from_angle_axis(ang, axis), frame_lab)
        np.testing.assert_allclose(ang, o_0.misorientation(o_i, me)[0])

In [ ]:
axis = np.array([1,1,1])
ang0 = np.random.random()*np.pi
ang1 = ang0 + np.pi/3
o_0 = Orientation(Quaternion.from_angle_axis(ang0, axis), frame_lab)
o_1 = Orientation(Quaternion.from_angle_axis(ang1, axis), frame_lab)

print()

# np.testing.assert_allclose(0, o_0.misorientation(o_1, 'cubic'))

## symmetry operator

In [13]:
GetSymRotMat??

Not much to do here, just need to map quaternion based symmetry operator to rotation matrix

## Orien2FZ

In [14]:
Orien2FZ??

In [16]:
to_fundamental_zone??

In [12]:
frame_lab = Frame()

o_0 = Orientation(Quaternion.from_eulers(np.radians([10, 90, 10])), frame_lab)

o_1 = to_fundamental_zone(o_0, 'hcp')

print(np.degrees(o_0.as_eulers.as_array))
print(np.degrees(o_1.as_eulers.as_array))


[170.  90. 190.]
[170.  90. 190.]
